# Create Pickel Model

In [ ]:
import pandas as pd
import numpy as np
from fbprophet import Prophet

In [ ]:
population_melt = pd.read_csv('csv/population_melt.csv')
population = pd.read_csv('csv/population_cleaned.csv')

In [ ]:
cities_list = list(population['City,State'])

In [ ]:
def rnd_series(city):
    subset = population_melt[population_melt['City,State']== city]
    dates = (pd.DataFrame({'ds': pd.to_datetime(population_melt['ds'])}))
    
    return subset

In [ ]:
series = [rnd_series(city) for city in cities_list]

In [ ]:
def make_model():
    series = [rnd_series(city) for city in cities_list]
    models = []
    for city_name in series:
        m = Prophet(interval_width=0.95)
        model = m.fit(city_name)
        models.append([city_name, model])

    # Putting information into dataframe
    model_df = pd.DataFrame(model, columns = ["cityname", 'model'])
    return model_df

In [ ]:
make_model()

In [ ]:
model_df = make_model()
print(model_df.shape)
model_df.head()

In [ ]:
import pickle

# save model_df in a pickle file
with open('pickle/model_df.pkl', 'wb') as f:
  pickle.dump(model_df, f)

In [ ]:
file = open("model_df.pkl", 'rb')
city_models = pickle.load(file)

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

In [ ]:
def run_forecast(city, periods):
    if city_name in list(city_models['cityname']): 
        city_model =  city_models.loc[city_model['cityname'] == city, 'model'].item()
        future = city_model.make_future_dataframe(periods=periods, freq='Y')
        forecast = city_model.predict(future)
        forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
        forecast['year'] = pd.DatetimeIndex(forecast['ds']).year
        forecast[['yhat', 'yhat_lower', 'yhat_upper']] = forecast[['yhat', 'yhat_lower', 'yhat_upper']].astype(float)
        forecast[['yhat', 'yhat_lower', 'yhat_upper']] = forecast[['yhat', 'yhat_lower', 'yhat_upper']].apply(np.ceil)
        forecast = forecast[['year', 'yhat', 'yhat_lower', 'yhat_upper']]
        fig1 = plot_plotly(city_model, forecast)
        fig2 = plot_components_plotly(city_model, forecast)
        return forecast, fig1, fig2

    return "This location is not available"